<a href="https://colab.research.google.com/github/Saoudyahya/COT-chain-of-thought-LLM/blob/main/COT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import userdata
access_token = userdata.get('HUGGINGFACE_ACCESS_TOKEN')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

model_name = "EleutherAI/gpt-neo-1.3B"  # Replace with the correct model name
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=access_token)

# Prepare the dataset
dataset = load_dataset("gsm8k",'main', split="train")  # Example dataset
def preprocess_function(examples):
    question = examples["question"]
    reasoning = examples["answer"]  # CoT reasoning
    input_text = f"Question: {question}\nAnswer:"
    target_text = reasoning
    return tokenizer(
        input_text,
        text_target=target_text,
        truncation=True,
        max_length=1000,  # Match expected length
        padding="max_length"  # Ensure consistent sequence lengths
    )

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./cot_model",
    evaluation_strategy="steps",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    fp16=True,  # Use mixed precision if available
    logging_dir='./logs'
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

def main():
    model_name = "EleutherAI/gpt-neo-1.3B"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=access_token)
    model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=access_token)

    # Add padding token if it doesn't exist
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.pad_token_id

    # Load and split dataset
    dataset = load_dataset("gsm8k", 'main')
    train_dataset = dataset["train"]
    eval_dataset = dataset["test"]  # GSM8K comes with a test split

    def preprocess_function(examples):
        # Combine question and answer for each example
        input_texts = [f"Question: {q}\nAnswer:" for q in examples["question"]]
        target_texts = examples["answer"]

        # Tokenize inputs
        model_inputs = tokenizer(
            input_texts,
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        # Tokenize targets using the text_target parameter
        labels = tokenizer(
            text_target=target_texts,
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # Process both datasets
    tokenized_train_dataset = train_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=train_dataset.column_names
    )

    tokenized_eval_dataset = eval_dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=eval_dataset.column_names
    )

    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./cot_model",
        evaluation_strategy="steps",
        eval_steps=500,  # Evaluate every 500 steps
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,  # Added eval batch size
        num_train_epochs=3,
        save_steps=500,
        save_total_limit=2,
        fp16=True,
        logging_dir='./logs',
        gradient_accumulation_steps=4,
        learning_rate=5e-5,
        # Add warmup steps
        warmup_steps=500,
        # Add weight decay
        weight_decay=0.01,
    )

    # Initialize trainer with both train and eval datasets
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,  # Added eval dataset
    )

    # Train the model
    trainer.train()

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: 

In [4]:
from huggingface_hub import login

# Replace 'your_token_here' with your actual token
login(token=access_token)


In [5]:
import transformers
import torch

model_id = "ContactDoctor/Bio-Medical-Llama-3-8B-CoT-012025"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are an expert trained on healthcare and biomedical reasoning."},
    {"role": "user", "content": "What are the differential diagnoses for a 45-year-old male presenting with chest pain?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/ContactDoctor/Bio-Medical-Llama-3-8B-CoT-012025.
403 Client Error. (Request ID: Root=1-67b06b93-1557afa972b3a96f78af5c0e;b6c6b00a-a892-4425-a3e3-4846393dc35e)

Cannot access gated repo for url https://huggingface.co/ContactDoctor/Bio-Medical-Llama-3-8B-CoT-012025/resolve/main/config.json.
Access to model ContactDoctor/Bio-Medical-Llama-3-8B-CoT-012025 is restricted and you are not in the authorized list. Visit https://huggingface.co/ContactDoctor/Bio-Medical-Llama-3-8B-CoT-012025 to ask for access.

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load a model fine-tuned for CoT reasoning
model_name = "google/flan-t5-large"  # Change model if you need a different one

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define a Chain-of-Thought prompt
prompt = (
    "Q: A farmer has 3 chickens, 2 cows, and 4 pigs. "
    "If each chicken has 2 legs, each cow has 4 legs, and each pig has 4 legs, "
    "how many total legs do all the animals have?\n"
    "Let's think step by step.\n"
    "A:"
)

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt")

# Generate a response
outputs = model.generate(inputs.input_ids, max_length=100, num_beams=5, early_stopping=True)

# Decode and print the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

First find the total number of chicken legs: 3 chickens * 2 legs / chicken = 6 legs. Then find the total number of cow legs: 2 cows * 4 legs / cow = 8 legs. Then find the total number of pig legs: 4 pigs * 4 legs / pig = 16 legs. Then add the number of each type of animal's legs to find the total number of legs: 6 legs + 8 legs + 16


In [7]:
# Simple CoT Prompt
prompt = (
    "Q: If a train travels 60 miles in one hour, how far will it travel in 3 hours?\n"
    "Let's think step by step.\n"
    "A:"
)

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt")

# Generate a response
outputs = model.generate(inputs.input_ids, max_length=100, num_beams=5, early_stopping=True)

# Decode and print the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

The train travels 60 miles / hour * 1 hour = 60 miles per hour. In 3 hours, the train will travel 60 miles / hour * 3 hours = 180 miles. The answer: 180.


In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# System prompt for context
system_prompt = (
    "You are talking to a 7-year-old child and explaining things like a parent. "
    "Keep your answers simple, kind, and easy to understand."
)

# Simple CoT Prompt with System Context
task_prompt = (
    "Q: If a train travels 60 miles in one hour, how far will it travel in 3 hours?\n"
    "Let's think step by step.\n"
    "A:"
)

# Combine the system and task prompts
full_prompt = f"{system_prompt}\n\n{task_prompt}"

# Tokenize the input prompt
inputs = tokenizer(full_prompt, return_tensors="pt")

# Generate a response
outputs = model.generate(inputs.input_ids, max_length=100, num_beams=5, early_stopping=True)

# Decode and print the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


In one hour, the train travels 60 miles / hour * 1 hour = 60 miles / hour. In 3 hours, the train travels 60 miles / hour * 3 hours = 180 miles / hour. The answer: 180.
